In [1]:
import sqlite3
import requests

conn = sqlite3.connect("2023DB_project.db")
cursor = conn.cursor()

- SQlite와 연동하기 위해 sqlite3, API에서 데이터를 가져오기 위해 requests 임포트
- connect 함수를 사용해 공영주차장과 관광지목록 테이블, 추천관광지 뷰가 있는 2023DB_project.db에 연결
- 쿼리를 실행하고 데이터를 조작하기 위해 커서 생성

In [2]:
key = "Hi4UuFuDdMfKYw5awA7KWXcLKxjXgu4JztYQQ7hpKB7DDhBhvmMzBIHXivGmgVcl"
endpoint = "https://incheon.openapi.redtable.global/api/rstr/korean"
url = f"{endpoint}?serviceKey={key}"

response = requests.get(url)

공공데이터포털의 오픈 API 음식점 정보 데이터베이스와 연결하기 위해 개인 인증키와 서비스 url을 얻어 get을 통해 연결

In [3]:
def parkinglot_nearby_spot(name):
    parkinglot_id=0
    if name=='월미테마파크':
        parkinglot_id=77
    elif name=='인천 차이나타운':
        parkinglot_id=76
    elif name=='소래포구':
        parkinglot_id=564
    elif name=='강화 전등사':
        parkinglot_id=675
    elif name=='송월동 동화마을':
        parkinglot_id=79
    elif name=='인천상륙작전기념관':
        parkinglot_id=678
    elif name=='짜장면박물관':
        parkinglot_id=76
    elif name=='수도국산 달동네박물관':
        parkinglot_id=680
    else:
        parkinglot_id=0
    return parkinglot_id

관광지와 가장 가까운 공영주차장 정보를 출력해주기 위해 위도와 경도를 이용해  가장 가까운 주차장을 찾고 objectid를 이용해 참조해주려고 했지만 데이터가 많아 구현하지 못했다. 추천관광지 목록을 선정하여 뷰로 만든 뒤 파이썬 함수에서 약식으로 구현했다.

In [4]:
spot_all_query = """
    SELECT 
        제목
    FROM 
        추천관광지
"""

spot_all_result = cursor.execute(spot_all_query)
print("인천광역시 이번 달의 추천관광지 목록입니다.")
for row in spot_all_result:
    print("-",row[0])

인천광역시 이번 달의 추천관광지 목록입니다.
- 강화 전등사
- 옥토끼우주센터
- 월미테마파크
- 인천 차이나타운
- 소래포구
- 송월동 동화마을
- 수도국산 달동네박물관
- 인천상륙작전기념관
- 짜장면박물관
- 강화자연사박물관


- 사용자가 처음 어플을 실행시키면 추천관광지 제목을 출력해 관광을 돕는 부분.
- 임의로 선정한 관광지를 추천관광지 뷰로 만들어 출력해준다.

In [5]:
name = input("알아보고 싶은 관광지를 입력하세요: ")

spot_query = f"""
    SELECT 
        제목, 
        주소, 
        전화번호, 
        애완동물동반가능정보, 
        주차시설, 
        쉬는날, 
        이용시간, 
        이용요금, 
        화장실설명 
    FROM 
        관광지목록 
    WHERE 
        제목 = '{name}'
"""

parking_lot = parkinglot_nearby_spot(name)

pLot_query = f"""
    SELECT 
        이름 
    FROM 
        공영주차장
    WHERE 
        ObjectID = {parking_lot}
"""

spot_result = cursor.execute(spot_query)

for row in spot_result:
    print("제목:", row[0])
    print("주소:", row[1])
    print("전화번호:", row[2])
    print("반려동물 동반 가능 정보:", row[3])
    print("주차시설:", row[4])
    print("쉬는 날:", row[5])
    print("이용시간:", row[6])
    print("이용요금:", row[7])
    print("화장실 설명:", row[8])

pLot_result = cursor.execute(pLot_query).fetchall()
if len(pLot_result) == 0:
    print("주변에 공영주차장이 없습니다.")
else:
    # 튜플의 첫 번째 요소를 출력
    print(f"인근 공영주차장: {pLot_result[0][0]}")

제목: 애관극장
주소: 인천 중구 개항로 63-2
전화번호: 032-761-7177
반려동물 동반 가능 정보: 불가
주차시설: 있음
쉬는 날: 없음
이용시간: 09:30~21:00
이용요금: 성인 7000원<br>
학생 6000원<br>
경로우대 및 조조 5000원
화장실 설명: 있음 (남녀구분, 건물 내부 위치)
주변에 공영주차장이 없습니다.


출력해준 추천관광지, 혹은 그 외에 정보를 알아보고 싶은 관광지의 이름을 입력하면 관광지에 대한 상세 정보와 주변 공영주차장 정보까지 출력해주는 부분.

In [ ]:
Rstr_name=input("주변의 음식점을 검색하기 위해 목적지의 군구정보를 입력해주세요(ex. 남동구, 강화군):")

if response.status_code == 200:
    try:
        response_body = response.json()['body']
        for item in response_body:
            if Rstr_name in item['LOTNO_ADDR']:
                print("가게 이름:",item['RSTR_NM'])
                print("도로명주소:",item['ROAD_NM_ADDR'])
                print("지번주소:",item['LOTNO_ADDR'])
    except KeyError as e:
        print(f'KeyError: {e}')
else:
    print(f'Error code: {response}')

관광지 주변에 있는 음식점들을 검색할 수 있도록 군이나 구 정보를 입력하면 인천시 내의 해당 군이나 구의 모든 음식점을 출력해준다.

In [ ]:
Rstr_name=input("상세정보를 알고싶은 음식점 이름을 정확히 입력해주세요:")

if response.status_code == 200:
    try:
        response_body = response.json()['body']
        for item in response_body:
            if item['RSTR_NM']==Rstr_name:
                print(item['RSTR_NM'])
                print(item['ROAD_NM_ADDR'])
                print(item['LOTNO_ADDR'])
                print(item['RSTR_RPRS_TELNO'])
                print(item['RSTR_EXPLN_CN'])
    except KeyError as e:
        print(f'KeyError: {e}')
else:
    print(f'Error code: {response}')

지역의 모든 음식점을 확인한 사용자가 그 중에서 원하는 음식점의 이름을 입력하면 그 음식점에 대한 상세정보를 출력해준다.

In [ ]:
conn.close()

코드 사용 후 2023DB_project.db와의 연결을 끊어준다